# 使用深度可分离卷积分类10-monkeys

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os,sys
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers
%matplotlib inline

In [2]:
import json
os.chdir('drive/MyDrive/Colab Notebooks')
!mkdir -p ~/.kaggle
!cp ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content/
!kaggle datasets download -d slothkong/10-monkey-species

- path is now set to: /content/
 95% 521M/547M [00:03<00:00, 176MB/s]
100% 547M/547M [00:03<00:00, 157MB/s]


In [5]:
%cd /content

/content


In [ ]:
!unzip -o -d /content datasets/slothkong/10-monkey-species/10-monkey-species.zip

In [7]:
train_dir = 'training/training/'
val_dir = 'validation/validation/'
label_file = 'monkey_labels.txt'
print(os.path.exists(train_dir))
print(os.path.exists(val_dir))
print(os.path.exists(label_file))

True
True
True


In [8]:
labels = pd.read_csv(label_file,header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [9]:
height,width = 128, 128 
channels,batch_size = 3, 64
num_classes = 10
# 进行数据增强
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255, # 把像素缩小到0-1之间
    rotation_range = 40, # 把图像岁间旋转0-40度之间
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True, # 水平翻转
) 

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(height,width),
                                                    batch_size=batch_size,seed=8,shuffle=True,
                                                    class_mode='categorical')

# 验证集只需要把像素缩小到0-1之间
val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir,target_size=(height,width),
                                               batch_size=batch_size,seed=8,shuffle=True,
                                                    class_mode='categorical')
train = train_generator.samples
val = val_generator.samples
print(train,val)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 272


In [13]:
# 开始搭建模型
model = keras.models.Sequential(
    [layers.Conv2D(32,(3,3),padding='same',activation='selu',input_shape=(width,height,channels)),
     # 采用深度可分离卷积
     layers.SeparableConv2D(32,(3,3),padding='same',activation='selu'),
     layers.SeparableConv2D(32,(3,3),padding='same',activation='selu'),
     layers.MaxPool2D(pool_size=2), # 经过池化之后，图片的宽度较小了，所以下一层要增加通道数来防止信息丢失
     layers.SeparableConv2D(64,(3,3),padding='same',activation='selu'),
     layers.SeparableConv2D(64,(3,3),padding='same',activation='selu'),
     layers.MaxPool2D(pool_size=2),
     layers.SeparableConv2D(128,(3,3),padding='same',activation='selu'),
     layers.SeparableConv2D(128,(3,3),padding='same',activation='selu'),
     layers.MaxPool2D(pool_size=2),
     layers.Flatten(), # 只有将图像展平之后才能输入全连接层
     layers.Dense(128,activation='selu'),
     layers.Dense(num_classes,activation='softmax') # 通过softmax函数激活输出类别的概率
     ]
)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,
          epochs=300,
          steps_per_epoch=train // batch_size,
          validation_data = val_generator,
          validation_steps = val //batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/300
17/17 [==============================] - 36s 2s/step - loss: 2.3171 - accuracy: 0.0851 - val_loss: 2.3132 - val_accuracy: 0.1172
Epoch 2/300
17/17 [==============================] - 34s 2s/step - loss: 2.3219 - accuracy: 0.1248 - val_loss: 2.3382 - val_accuracy: 0.0977
Epoch 3/300
17/17 [==============================] - 33s 2s/step - loss: 2.3417 - accuracy: 0.0861 - val_loss: 2.3117 - val_accuracy: 0.0977
Epoch 4/300
17/17 [==============================] - 33s 2s/step - loss: 2.3033 - accuracy: 0.1132 - val_loss: 2.2446 - val_accuracy: 0.1680
Epoch 5/300
17/17 [==============================] - 33s 2s/step - loss: 2.1496 - accuracy: 0.2253 - val_loss: 2.0302 - val_accuracy: 0.2344
Epoch 6/300
17/17 [==============================] - 33s 2s/step - loss: 2.1186 - accuracy: 0.2137 - val_loss: 2.0360 - val_accuracy: 0.2930
Epoch 7/300
17/17 [==============================] - 33s 2s/step - loss: 2.1054 - accuracy: 0.2418 - val_loss: 1.9307 - val_accuracy: 0.3555
Epoch 8/300
1